## Title: Holdridge Life-Zones
### Description
Holdridge's work aimed to correlate world plant formations with simple climatic data. The system embraces all major environmental factors in three hierarchical tiers. Level I - The Life Zone. This is determined by specific quantitative ranges of long-term average annual precipitation, mean annual biotemperature and potential evapotranspiration ratio. These are modified for montane systems. Level II - The "Association". This is an area of land which, under undisturbed conditions, supports a distinctive natural community adapted through evolution to a specific narrow range of atmospheric and edaphic conditions. No Association can occur in more than one Life Zone. Level III - The successional stage or cover type, which takes into account that the community may not be in its climax state, either through natural causes or through human intervention.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/<br>

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Year: </b>2010<br>
<b>Size:</b> 2 MB <br>

### Original source
Access: http://www.fao.org/geonetwork/srv/en/metadata.show?id=1006 Accessed 13/12/2020

### Licence
Access to UNEP-WCMC datasets is provided on the understanding that you read and consent to be bound by our Terms and Conditions. For the purposes of this Agreement the “Data” comprise any of the spatial data and associated attribute data downloadable from the UNEP-WCMC website, excluding the World Database on Protected Areas. 
1) Adhere to terms and conditions of access, 2) acknowledge the source appropriately, 3) do not re-distribute part or all of data without contacting UNEP-WCMC.
Additional permission info: [Github](https://github.com/moja-global/Land-Sector-Datasets/blob/master/Additional-Permissions/Gmail%20-%20Thank%20you%20for%20useful%20input%20and%20follow%20up.pdf)

### Citation
Holdridge, L.R. collected and compiled by Leemans, R. 1990 <br>
Leemans, R. 1992. Global Holdridge Life Zone Classifications.

### Original Format
Global coverage, vector, shapefile <br>
Cordinate system EPSG: Unknown

### Metadata
http://www.fao.org/geonetwork/srv/en/metadata.show?id=1006<br>
Date created: 1989<br>
Vector map based on 0.5-degree cells (360 x 720).

### Notes
Missing coordinate system defined to 4326, Zone 0 is water and has a blank description (DESC).<br>
Original map prepared at a very general scale.<br>
Anctarctica continent not classified.<br>
No topological errors in the original.<br>

### Processing
Repair geometry, convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
# Import arcpy module
import arcpy
import os

# Input local variables
in_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\holdrid"
scr_folder = r"C:\Data\scratch.gdb"
out_folder = r"C:\Data\json"

# Environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True
scr = arcpy.CreateFileGDB_management(r"C:\Data", "scratch")

# List features to process
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Repair/check topology and make FLINT ready
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)  

    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(projectSingle, dissolveSlither, None, None,"SINGLE_PART")
    if arcpy.management.GetCount(dissolveSlither)[0] == "0":
        print('no overlaps detected')
    else:
        print('error')
        
    projectUnion = os.path.join(scr_folder, "projectUnion")
    arcpy.analysis.Union(fLayer,projectUnion, "ALL", None, "NO_GAPS")
    
    whereclause = "FID_" + fcname + "= -1"
    uniSelect = os.path.join(scr_folder, "uniSelect")
    arcpy.analysis.Select(projectUnion, uniSelect, whereclause)

    if arcpy.management.GetCount(uniSelect)[0] == "0":
        print('no gaps detected')
    else:
        print('error')
        
# Progress report 
print(fcname, 'gaps and overlaps checked. Repairing geometry and conversion to json...')

# Process: Repair Geometry (non-simple geometry)
geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

# Process: Features To JSON
arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
    
print(outjson, '.geojson complete')
    
arcpy.AddMessage("All done!")
print('done')